## 더 나은 성능의 tf.function
##### url: https://www.tensorflow.org/tutorials/customization/performance

- 목차
    - 준비
    - 기본
    - 추적과 다형성
    - 재추적 시기
    - 파이썬 또는 텐서 인자
    - tf.function의 부작용
    - 파이썬 상태 값의 주의점
    - 자동 제어 의존성
    - 변수
    - AutoGraph 사용하기
    - AutoGraph: 조건
    - AutoGraph와 반복문
        - 루프
        - 찾은 것
    - 더 읽어볼 내용
    
Tensorflow 2.0에서는 즉시 실행 기능을 기본으로 채택하고 있습니다. 유저 인터페이스는 직관적이고 유연(하나의 연산을 실행하는것이 더욱 쉽고 빠릅니다.)하지만, 배포성과 성능 측면에서 비용을 지불하고 있습니다.<br><br>
최대 성능을 얻는 것과 어디에서든 배포가 가능한 모델을 만들기 위해서는 `tf.function`을 사용하여 프로그램의 그래프를 만들어야합니다. 고맙게도 AutoGraph는, 놀라운 양의 파이썬 코드로 tf.function과 함께 작동하지만 여전히 주의해야할 어려움이 존재합니다.<br><br>
다음은 피해야하는 것과 추천되는 것들 입니다.
- 변환 가능한 객체와 리스트 어펜드 같은 파이썬 부작용에 의존하지마라 
- tf.function은 Tensorflow 연산을 NumPy나 원시 파이썬 연산보다 가장 잘(best) 작동시킨다.
- `for x in y` 같은 관용구를 사용할 때 의심해라.

## 준비

In [1]:
import tensorflow as tf

직면할 수 있는 에러들을 시연하기 위해 helper 함수를 정의하겠습니다.

In [2]:
import traceback
import contextlib

# 직면할 수 있는 에러를 시연하기 위한 몇몇의 helper 코드
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('예상되던 예외를 찾았습니다. \n {}:'.format(error_class))
        traceback.print_exc(limit=2)
    except Exception as e:
        raise e
    else:
        raise Exception('예상된 {}가 발생하였으나 에러는 발생하지 않았습니다.'.format(error_class))